In [1]:
!pip install pyod --user

In [3]:
!pip install seaborn 

  Using cached seaborn-0.11.2-py3-none-any.whl (292 kB)


In [9]:
import pandas as pd
import seaborn as sns
from pyod.models.knn import KNN
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

from sklearn.decomposition import PCA
import sklearn.neighbors
from sklearn.neighbors import kneighbors_graph
from sklearn import preprocessing



In [5]:
df = pd.read_csv('data/suelos_preprocesado.csv')
df.head(2)

,id,departamento,municipio,cultivo,estado,tiempo_establecimiento,topografia,drenaje,riego,fertilizantes,...,ce,hierro_olsen,cobre,manganeso,zinc_olsen,boro,hierro_doble_acido,cobre_doble_acido,manganeso_doble_acido,zinc_doble_acido
0,NaN,cundinamarca,funza,uchuva,por establecer,no aplica,ondulado,bueno,no indica,no indica,...,0.223513,184.118,1.000,2.876,3.462,0.109442,NaN,NaN,NaN,NaN
1,NaN,cundinamarca,bituima,citricos,por establecer,no aplica,ondulado,bueno,no cuenta con riego,no indica,...,0.692732,33.181,3.426,1.000,90.926,0.710300,NaN,NaN,NaN,NaN


In [9]:
# Variables Numericas
numerics_variables=df[['ph', 'materia_organica', 'fosforo', 'azufre','acidez', 'aluminio', 'calcio', 'magnesio', 'potasio', 'sodio', 'cice',
       'ce', 'hierro_olsen', 'cobre', 'manganeso', 'zinc_olsen', 'boro','hierro_doble_acido', 'cobre_doble_acido', 'manganeso_doble_acido',
       'zinc_doble_acido']]

In [6]:
new_df=df.copy()

In [7]:
Y=new_df[['ph', 'materia_organica', 'fosforo', 'calcio', 'magnesio', 'potasio','sodio', 'cice']].fillna(0)

In [10]:
# Se normalizan los datos con MinMax()
min_max_scaler = preprocessing.MinMaxScaler() 
df_escalado = min_max_scaler.fit_transform(Y)
df_escalado = pd.DataFrame(df_escalado) 
df_escalado = df_escalado.rename(columns = {0: 'ph', 1: 'materia_organica', 2: 'fosforo' , 3: 'calcio', 4: 'magnesio', 5: 'potasio', 6: 'sodio', 7: 'cice'})

In [13]:
#Forma 1, detecta outliers para un congunto de variables
clf = KNN(contamination=0.005)
modelo=clf.fit(df_escalado)
y_pred = clf.predict(df_escalado)
Y[y_pred == 1]

,ph,materia_organica,fosforo,calcio,magnesio,potasio,sodio,cice
1676,5.60,2.053724,25.652770,6.904364,12.923008,0.504274,7.847250,28.178895
1930,8.63,2.001435,142.908096,15.767601,12.270756,0.828152,4.767276,33.633785
2247,3.80,43.493217,7.218225,0.813324,0.713982,0.571739,0.106790,15.088271
2496,6.23,20.306435,296.885554,20.273931,7.771437,7.555836,0.679782,36.280986
3054,8.58,12.904946,419.994477,29.538890,17.375078,42.897035,10.354806,100.165810
...,...,...,...,...,...,...,...,...
44272,6.15,14.840000,204.420000,25.530000,10.490000,8.800000,0.670000,45.610000
44308,3.87,0.680000,9.210000,4.960000,7.510000,0.480000,3.640000,26.570000
44311,3.99,1.290000,10.840000,2.200000,7.840000,0.440000,2.840000,21.230000
44456,6.27,24.820000,13.640000,31.250000,3.030000,2.320000,0.170000,36.770000


In [14]:
y_pred_df = pd.DataFrame(y_pred, columns = ['Prediction'])
y_pred_df['Prediction'].unique()

array([0, 1])

In [ ]:
data = pd.concat([df_escalado, y_pred_df], axis = 1)
# Grafico
sns.pairplot(data=data, hue="Prediction")

2. IQR (Inter Quartile Range

In [28]:
def detect_otliers(x): #funcion para detectar outliers, esto no hace parte de este ejercicio
    Q3 = Y[x].quantile(0.75)
    Q1 = Y[x].quantile(0.25)
    IQR = Q3 - Q1
    superior = Q3 + (1.5 * IQR)
    inferior = Q1 - (1.5 * IQR)
    out_sup = Y[Y[x] > superior].index
    out_inf = Y[Y[x] < inferior].index
    outliers = []
    for i in out_sup:
        outliers.append(i)
        for j in out_inf:
            outliers.append(j)
    
    size = len(outliers)
        
    return (f'Hay {size} valores atípicos en la variable {x}, y corresponden a los índices: {outliers}')



In [ ]:
detect_otliers('materia_organica')

'Hay 3754 valores atípicos en la variable materia_organica, y corresponden a los índices: [4, 49, 88, 89, 90, 94, 96, 100, 103, 105, 114, 136, 142, 143, 144, 145, 146, 164, 167, 169, 171, 181, 183, 189, 190, 191, 192, 195, 198, 199, 201, 203, 205, 207, 213, 215, 218, 219, 221, 227, 229, 257, 259, 260, 261, 262, 263, 265, 266, 267, 268, 269, 270, 272, 276, 277, 279, 281, 282, 283, 284, 285, 287, 288, 289, 290, 291, 292, 293, 294, 295, 297, 419, 424, 425, 426, 427, 429, 432, 436, 438, 442, 444, 499, 500, 501, 502, 503, 508, 519, 520, 521, 524, 526, 528, 560, 562, 567, 574, 576, 581, 589, 764, 779, 782, 787, 788, 806, 847, 880, 881, 925, 926, 930, 931, 932, 950, 979, 1021, 1023, 1028, 1096, 1097, 1098, 1099, 1102, 1110, 1117, 1123, 1125, 1127, 1129, 1134, 1211, 1230, 1247, 1248, 1250, 1264, 1269, 1270, 1271, 1272, 1273, 1275, 1276, 1278, 1304, 1305, 1316, 1318, 1320, 1324, 1326, 1327, 1329, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1339, 1341, 1366, 1374, 1381, 1385, 1389, 1390, 1417, 148

In [32]:
from sklearn.cluster import KMeans
from sklearn import metrics


In [45]:
model = KMeans(n_clusters = 20, max_iter = 1000) # we can evaluate the model with different n_cluster in order to determine the corrected one.
model.fit(Y)
y_labels = model.labels_

In [46]:
y_kmeans = model.predict(Y)

#print(accuracy)

In [53]:
y_kmeans_df = pd.DataFrame(y_kmeans, columns = ['Prediction'])
data = pd.concat([Y, y_kmeans_df], axis = 1)
# Grafico
#sns.pairplot(data=data, hue="Prediction")
data.head()

,ph,materia_organica,fosforo,calcio,magnesio,potasio,sodio,cice,Prediction
0,5.66,9.706679,5.622299,4.965692,0.978829,0.204534,0.145927,6.294983,0
1,8.08,3.417595,7.570468,42.704797,1.275356,0.131217,0.120282,44.231652,13
2,5.87,2.336659,16.901172,9.597183,0.945701,0.194181,0.138463,10.875528,9
3,5.56,6.853757,32.076383,7.528300,0.841608,1.590415,0.140000,10.060324,9
4,4.87,16.041251,64.199200,2.234775,0.346672,0.313188,0.156357,8.079447,5


In [54]:
from sklearn.cluster import DBSCAN

Z- Score is also called a standard score. This value/score helps to understand that how far is the data point from the mean. And after setting up a threshold value one can utilize z score values of data points to define the outliers.

Zscore = (data_point -mean) / std. deviation

In [57]:
from scipy import stats
z = np.abs(stats.zscore(Y['ph']))
threshold = 3
 # Position of the outlier
print(np.where(z > 3))

(array([ 1428,  1930,  3054,  4171,  4460,  4465,  5164,  5320,  6460,
        6662,  9064,  9968, 10776, 10819, 11180, 12438, 12440, 12441,
       12442, 12443, 12444, 12445, 12446, 12447, 12450, 12451, 12452,
       12453, 12458, 12462, 13318, 14977, 15371, 15387, 15779, 16113,
       16642, 19430, 19784, 20375, 24802, 25523, 25524, 25525, 25526,
       25527, 25529, 25530, 25560, 25561, 25562, 25794, 25795, 25802,
       25803, 25814, 25815, 26674, 26676, 26677, 26680, 26684, 26688,
       26689, 26690, 26691, 26692, 26693, 26695, 26697, 26698, 26702,
       26789, 26816, 27559, 30853, 30987, 30989, 30990, 30991, 30996,
       35352, 36786, 36788, 37466, 38364, 39628, 40299, 41165, 42332],
      dtype=int64),)


DBSCAN

In [106]:
# Se normalizan los datos con MinMax()
min_max_scaler = preprocessing.MinMaxScaler() 
df_escalado = min_max_scaler.fit_transform(Y)
df_escalado = pd.DataFrame(df_escalado) 
df_escalado = df_escalado.rename(columns = {0: 'ph', 1: 'materia_organica', 2: 'fosforo' , 3: 'calcio', 4: 'magnesio', 5: 'potasio', 6: 'sodio', 7: 'cice'})

In [107]:
df_escalado.head()

,ph,materia_organica,fosforo,calcio,magnesio,potasio,sodio,cice
0,0.325513,0.143815,0.001864,0.035999,0.032554,0.002318,0.004061,0.043538
1,0.680352,0.051586,0.002510,0.311162,0.042479,0.001487,0.003347,0.314353
2,0.356305,0.035734,0.005605,0.069768,0.031445,0.002200,0.003853,0.076236
3,0.310850,0.101977,0.010637,0.054683,0.027962,0.018021,0.003896,0.070417
4,0.209677,0.236712,0.021289,0.016087,0.011397,0.003549,0.004351,0.056276


In [135]:
## Parametrización de DBSCAN.
dbscan = DBSCAN(eps=0.2, min_samples = 10, metric = "euclidean").fit(df_escalado)
clusters = dbscan.fit_predict(df_escalado)
y_pred_df = pd.DataFrame(clusters, columns = ['Prediction'])

#grafica
#data = pd.concat([df_escalado, y_pred_df], axis = 1)
# Grafico
#sns.pairplot(data=data, hue="Prediction")


MemoryError: 